# Air Quality and Respiratory Disease
## Step1. Data Cleaning

### **Data:**

In this project, we will be using three different data. Datasset can be found here:

In [2]:
# Import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from functools import partial
import plotly.subplots as sp
import scipy.stats as stats
from tqdm.notebook import tnrange 
from time import sleep
from IPython.display import clear_output

In [27]:
d = {"Province": [],
    "Pollutant": []}
    
provinces = ["AB", "BC", "ON", "MB", "NB", "NS", "YT", "QC", "SK", "NL", "PE"]
common_pollutants = ["NH3","CO","SOX","NOX","VOC"]
provnum = 0

def import_data(province, pollutants):
    df = pd.DataFrame.from_dict(d)
    for i in pollutants:
        p = pd.read_csv("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/province-pollutants/" + province + "_" + i + ".csv")
        p_total = p.loc[p["SECTORS"] == "GRAND TOTAL"]
        p_total = p_total.drop(["Unnamed: 1", "Unnamed: 2"], axis=1)
        p_total = p_total.rename(columns={"SECTORS":"Province"})
        p_total.at[1,"Province"] = province
        p_total.insert(1,"Pollutant", i, True)
        df = pd.concat([df, p_total], ignore_index=True)
        
        for i in tnrange(100, desc='Downloading ' + i + ' data for ' + province):
            startRow = 1 + i*100
            sleep(0.01)
    
    global provnum
    if provnum > 11:
        provnum = 0
    else:
        provnum += 1
            
    print("All provincial data for " + province + " has been downloaded." + 
         " Remaining provinces:" + str(len(provinces) - provnum) + "/" + str(len(provinces)) + ".") 
        
    return df

df = pd.concat(map(partial(import_data, pollutants=common_pollutants), provinces),ignore_index=True)
clear_output(wait=True)
df.head()

,Province,Pollutant,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AB,NH3,95261.6403072018,96392.6120160835,100809.748597631,102888.757346287,108909.29619548,117873.246725767,123629.070648118,126274.333185743,...,132641.48219304642,137630.5644576695,139707.6493382899,141095.12881051924,139937.588499843,132619.48058146506,124020.85788901642,130492.08344071463,131440.5311861856,133161.78812024335
1,AB,CO,1810389.43111,1674240.47593966,1694347.04215079,1670384.18553975,1713860.13181427,1699241.95442997,1707362.43286568,1747506.53522988,...,1015160.0103461307,1020173.0616697504,1044806.4504109875,1083451.1163415606,985516.9393906805,950437.9647403281,1009269.6788974762,1001193.5531940853,992062.7619323295,890235.0708604576
2,AB,SOX,512405.17695316,524144.18481018,564717.776802336,571473.902885045,594900.160427548,569212.579973766,556679.129617835,522989.264001654,...,343134.3155222252,333877.81221442844,314187.69199869514,291592.47385681805,259725.37296679677,239568.79130380912,241439.9996413067,225758.93672193997,221287.8184768282,182536.32978225523
3,AB,NOX,613284.286308138,585705.325771787,609463.591567837,640884.40812074,692547.095770932,718124.92776215,749396.467537041,815873.317241737,...,691938.4815302572,649050.7227761747,643710.2914325446,656735.6508195029,630979.6246838769,603268.9203695664,627271.8514718515,625976.446078691,625636.7223761571,567697.196882858
4,AB,VOC,643752.225882095,624633.776990753,642477.562274331,654384.418252455,669557.452773095,679792.446419822,708522.408799455,678362.261827271,...,478680.32202698546,518824.5195309372,556411.0771793935,572615.3277760863,517421.8223597623,476740.12370002636,474301.4225637558,501074.8373153089,492636.6127706006,456932.64826396375


In [50]:
# data cleaning for air pollutant data
per_capita = pd.read_csv("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/per-capita.csv")
air_pollutant = df.melt(id_vars = ["Province", "Pollutant"], var_name = "Year", value_name = "Value")
air_pollutant["Year"] = air_pollutant["Year"].astype(int)

# data cleaning for per capita data
prov = {"Canada": "CAN", "Newfoundland and Labrador": "NL", "Prince Edward Island":"PE",
       "Nova Scotia":"NS", "New Brunswick":"NB", "Quebec":"QC", "Ontario":"ON",
       "Manitoba":"MB", "Alberta":"AB", "Saskatchewan":"SK", "British Columbia":"BC",
       "Yukon":"YK", "Northwest Territories":"NT", "Nunavut":"NU"}
per_capita = per_capita.replace(prov)
per_capita["Year"]=pd.to_datetime(per_capita["REF_DATE"]).dt.year
per_capita=per_capita.groupby(["Year","GEO"], as_index=False).mean()
per_capita = per_capita[["Year", "GEO", "VALUE"]]
per_capita = per_capita.loc[per_capita["Year"] < 2021]
per_capita = per_capita.rename(columns= {"GEO":"Province", "VALUE":"Per Capita"})

# merge the air pollutant and per capita datsets
pollutant_capita = pd.merge(air_pollutant, per_capita, on=["Year", "Province"])

# divide 
pollutant_capita["Value"] = pd.to_numeric(pollutant_capita["Value"])
pollutant_capita["By Capita"] = pollutant_capita["Value"].div(pollutant_capita["Per Capita"])
pollutant_capita.head()

,Province,Pollutant,Year,Value,Per Capita,By Capita
0,AB,NH3,1990,9.526164e+04,2540901.75,0.037491
1,AB,CO,1990,1.810389e+06,2540901.75,0.712499
2,AB,SOX,1990,5.124052e+05,2540901.75,0.201663
3,AB,NOX,1990,6.132843e+05,2540901.75,0.241365
4,AB,VOC,1990,6.437522e+05,2540901.75,0.253356


In [49]:
# calculate mean per capita emission by pollutants. 
pollutant_capita_mean = pollutant_capita[["Province", "Pollutant", "By Capita"]].groupby(["Province", "Pollutant"]).mean()
pollutant_capita_mean = pollutant_capita_mean.rename(columns={"By Capita": "Mean"}).reset_index()
pollutant_capita_mean

# calculate total emission of each pollutants from all provinces.  
pollutant_capita_sum = pollutant_capita[["Province", "Pollutant", "Value"]].groupby(["Province", "Pollutant"]).sum()
pollutant_capita_sum = pollutant_capita_sum.rename(columns={"Value": "Total Emission"}).reset_index()
pollutant_capita_sum

pollutant_capita_mean.head()

,Province,Pollutant,Mean,Per Capita
0,AB,CO,0.426834,3.398268e+06
1,AB,NH3,0.038578,3.398268e+06
2,AB,NOX,0.209131,3.398268e+06
3,AB,SOX,0.131820,3.398268e+06
4,AB,VOC,0.179616,3.398268e+06


In [1]:
can_respiratory = pd.read_csv("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/canadian-respiratory-death.csv")
can_respiratory = can_respiratory.loc[can_respiratory["Cause of death (ICD-10)"] == "Chapter X: Diseases of the respiratory system [J00-J98]"]
can_respiratory = can_respiratory[["REF_DATE", "VALUE"]].rename(columns={"REF_DATE":"Year",
                                                                        "VALUE":"Death"})
can_respiratory.head()

NameError: name 'pd' is not defined

In [41]:
comb_df = pd.merge(pollutant_capita, can_respiratory, on="Year", how="inner")
comb_df["Death per Capita"] = comb_df["Death"] / comb_df["Per Capita"]
comb_df

,Province,Pollutant,Year,Value,Per Capita,By Capita,Death,Death per Capita
0,AB,NH3,2000,1.399578e+05,2996031.75,0.046714,17745,0.005923
1,AB,CO,2000,1.677588e+06,2996031.75,0.559937,17745,0.005923
2,AB,SOX,2000,4.907949e+05,2996031.75,0.163815,17745,0.005923
3,AB,NOX,2000,8.018365e+05,2996031.75,0.267633,17745,0.005923
4,AB,VOC,2000,6.306122e+05,2996031.75,0.210482,17745,0.005923
...,...,...,...,...,...,...,...,...
1045,PE,NH3,2020,3.203449e+03,160633.75,0.019943,23255,0.144770
1046,PE,CO,2020,2.030025e+04,160633.75,0.126376,23255,0.144770
1047,PE,SOX,2020,2.709045e+02,160633.75,0.001686,23255,0.144770
1048,PE,NOX,2020,3.301916e+03,160633.75,0.020556,23255,0.144770


In [43]:
comb_df.to_csv("cleaned-data/comb_df.csv")